In [1]:
from tinybig.config import rpn_config
from tinybig.util import set_random_seed

print('loading configs...')
config_file_name = 'cifar10_configs'
config_obj = rpn_config()
config = config_obj.load_yaml(cache_dir='./configs', config_file=config_file_name + '.yaml')
print(config)

loading configs...
{'configurations': {'device': 'mps', 'random_seed': 1234, 'data_configs': {'data_class': 'tinybig.data.cifar10', 'data_parameters': {'name': 'mnist', 'train_batch_size': 100, 'test_batch_size': 64}}, 'learner_configs': {'learner_class': 'tinybig.learner.backward_learner', 'learner_parameters': {'name': 'error_backward_propagation', 'n_epochs': 11, 'optimizer_configs': {'optimizer_class': 'torch.optim.AdamW', 'optimizer_parameters': {'lr': 0.001, 'weight_decay': 1e-05}}, 'lr_scheduler_configs': {'lr_scheduler_class': 'torch.optim.lr_scheduler.ExponentialLR', 'lr_scheduler_parameters': {'gamma': 0.65}}, 'loss_configs': {'loss_class': 'torch.nn.CrossEntropyLoss', 'loss_parameters': {'reduction': 'mean'}}}}, 'model_configs': {'model_class': 'tinybig.model.rpn', 'model_parameters': {'name': 'reconciled_polynomial_network', 'depth': 3, 'depth_alloc': [1, 1, 1], 'layer_configs': [{'layer_class': 'tinybig.module.rpn_layer', 'layer_parameters': {'name': 'rpn_layer', 'm': 3072

In [2]:
print('setting up environments...')
device = config['configurations'].pop('device')
random_seed = config['configurations'].pop('random_seed')
set_random_seed(random_seed)
print('device: ', device, '; random_seed: ', random_seed)

setting up environments...
device:  mps ; random_seed:  1234


In [3]:
print('instantiating objects from config...')
object_dict = config_obj.instantiate_object_from_config(config['configurations'])
data_obj, model_obj, learner_obj, metric_obj, result_obj = [object_dict[name] for name in ['data', 'model', 'learner', 'metric', 'result']]
print('parameter num: ', sum([parameter.numel() for parameter in model_obj.parameters()]))

instantiating objects from config...
parameter num:  1706496


In [4]:
print('loading dataset...')
data_loader = data_obj.load()

loading dataset...
Files already downloaded and verified
Files already downloaded and verified


In [5]:
print('training model...')
training_records = learner_obj.train(model=model_obj, data_loader=data_loader, device=device, metric=metric_obj)
model_obj.save_ckpt(cache_dir='./ckpt', checkpoint_file=f'{config_file_name}_checkpoint')

training model...


100%|██████████| 500/500 [00:07<00:00, 65.77it/s, epoch=0/11, loss=1.59, lr=0.001, metric_score=0.42, time=7.62]


Epoch: 0, Test Loss: 1.4929834885202395, Test Score: 0.4674, Time Cost: 1.3373339176177979


100%|██████████| 500/500 [00:07<00:00, 66.27it/s, epoch=1/11, loss=0.999, lr=0.00065, metric_score=0.64, time=16.5]


Epoch: 1, Test Loss: 1.384826884907522, Test Score: 0.5077, Time Cost: 1.2460088729858398


...

Epoch: 8, Test Loss: 1.3010211503429778, Test Score: 0.5641, Time Cost: 1.17451810836792


100%|██████████| 500/500 [00:07<00:00, 67.09it/s, epoch=9/11, loss=0.788, lr=2.07e-5, metric_score=0.69, time=86.2]


Epoch: 9, Test Loss: 1.3110735256960437, Test Score: 0.5632, Time Cost: 1.1685810089111328


100%|██████████| 500/500 [00:07<00:00, 66.82it/s, epoch=10/11, loss=0.705, lr=1.35e-5, metric_score=0.79, time=94.8]


Epoch: 10, Test Loss: 1.310013646153128, Test Score: 0.5672, Time Cost: 1.1657207012176514
model checkpoint saving to ./ckpt/cifar10_configs_checkpoint...


In [6]:
print('testing model...')
test_result = learner_obj.test(model=model_obj, test_loader=data_loader['test_loader'], device=device,
                               metric=metric_obj)

testing model...


In [7]:
print('evaluating result...')
print(metric_obj.__class__.__name__, metric_obj.evaluate(y_true=test_result['y_true'], y_pred=test_result['y_pred'], y_score=test_result['y_score'], ))

evaluating result...
accuracy 0.5672


In [8]:
print('saving result...')
result_obj.save(test_result, cache_dir='./result', output_file='{}_result'.format(config_file_name))
result_obj.save(training_records, cache_dir='./result', output_file='{}_record'.format(config_file_name))

saving result...
